# Part 3(b)

This notebook will highlight the process, result and insights obtained from using the embedding matrix built that includes OOV words with their contextual average as embeddings.

## Import Embedding Matrix

In [1]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/w2v_matrix.npy")
index_from_word_path = Path("models/w2v_index.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare Dataaset

(a) Import datasets

In [2]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juinl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\juinl\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\juinl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

train_dataset

Dataset({
    features: ['text', 'label', 'tokens', 'original_len', 'indexes'],
    num_rows: 8530
})

In [4]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [5]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

## Train RNN

We use Optuna to perform heuristic search on optimal configuration when the embedding matrix with OOV words is updatable during training.

In [8]:
import optuna
from utils.train import train_rnn_model_with_parameters

SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-1, 1e-2, 1e-3, 1e-4],
    "optimizer_name": ["Adam"],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])
    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        freeze_embedding=False,
        log_dir="rnn/test/w2v-3b"
    )
    
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=500) 

best_params = study.best_params


[I 2024-11-04 00:28:03,558] A new study created in memory with name: no-name-5f0b6e84-68a5-43ef-bce2-e86bdcd43616
Seed set to 42


---------- batch_size_512; lr_0.01; optimizer_Adam; hidden_dim_32; num_layers_2; sentence_representation_last ----------


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 4.9 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.654    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

[Skipping] rnn/test/w2v-3b/batch_size_2048-lr_0.001-optimizer_Adam-hidden_dim_32-num_layers_1-sr_type_max-freeze_False
---------- batch_size_2048; lr_0.001; optimizer_Adam; hidden_dim_32; num_layers_1; sentence_representation_max ----------


[I 2024-11-04 02:24:46,380] Trial 498 finished with value: 0.7804877758026123 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'optimizer_name': 'Adam', 'batch_size': 2048, 'learning_rate': 0.001, 'sentence_representation_type': 'max'}. Best is trial 50 with value: 0.7804877758026123.
Seed set to 42


[Skipping] rnn/test/w2v-3b/batch_size_2048-lr_0.001-optimizer_Adam-hidden_dim_32-num_layers_1-sr_type_max-freeze_False
---------- batch_size_2048; lr_0.001; optimizer_Adam; hidden_dim_32; num_layers_1; sentence_representation_max ----------


[I 2024-11-04 02:24:47,034] Trial 499 finished with value: 0.7804877758026123 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'optimizer_name': 'Adam', 'batch_size': 2048, 'learning_rate': 0.001, 'sentence_representation_type': 'max'}. Best is trial 50 with value: 0.7804877758026123.


[Skipping] rnn/test/w2v-3b/batch_size_2048-lr_0.001-optimizer_Adam-hidden_dim_32-num_layers_1-sr_type_max-freeze_False


## Train Result Analysis

In [6]:
print("Best hyperparameters:", best_params)

NameError: name 'best_params' is not defined

(a) Load result from all trials

In [17]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(log_dir="tb_logs/rnn/test/w2v-3b")

train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
train_results_df.head(20)

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,freeze,epoch,train_loss,val_loss,filename
0,0.777674,0.925366,2048,32,0.0001,Adam,1,last,False,67.0,0.329988,0.512574,events.out.tfevents.1730657533.LEEJUIN-PC.3436...
1,0.776735,NaN,2048,256,0.0010,Adam,2,last,False,8.0,NaN,0.511208,events.out.tfevents.1730656882.LEEJUIN-PC.3436...
2,0.770169,NaN,2048,256,0.0010,Adam,3,max,False,8.0,NaN,0.502858,events.out.tfevents.1730655326.LEEJUIN-PC.3436...
3,0.769231,NaN,2048,256,0.0010,Adam,1,max,False,7.0,NaN,0.477541,events.out.tfevents.1730656014.LEEJUIN-PC.3436...
4,0.768293,NaN,2048,32,0.0100,Adam,1,max,False,5.0,NaN,0.492991,events.out.tfevents.1730654123.LEEJUIN-PC.3436...
5,0.768293,NaN,2048,128,0.0010,Adam,1,max,False,8.0,NaN,0.477828,events.out.tfevents.1730651792.LEEJUIN-PC.3436.72
6,0.768293,0.887246,2048,32,0.0001,Adam,1,max,False,75.0,0.403241,0.529162,events.out.tfevents.1730654282.LEEJUIN-PC.3436...
7,0.768237,0.846502,1024,32,0.0010,Adam,1,max,False,10.0,0.471649,0.505253,events.out.tfevents.1730655743.LEEJUIN-PC.3436...
8,0.763602,0.871323,2048,32,0.0010,Adam,2,max,False,13.0,0.415798,0.520513,events.out.tfevents.1730654724.LEEJUIN-PC.3436...
9,0.763602,0.846154,2048,256,0.0100,Adam,1,average,False,10.0,0.346450,0.576905,events.out.tfevents.1730654936.LEEJUIN-PC.3436...


(b) Configuration for best trial result

In [18]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,freeze,epoch,train_loss,val_loss,filename
0,0.777674,0.925366,2048,32,0.0001,Adam,1,last,False,67.0,0.329988,0.512574,events.out.tfevents.1730657533.LEEJUIN-PC.3436...


## Performance on Test Dataset

In [19]:
from pathlib import Path
from utils.train import RNNClassifier


best_rnn_model_filename = best_rnn_model_configuration["filename"].item()
matched_files = list(Path().rglob(best_rnn_model_filename))

if not matched_files:
    print("Model checkpoint not found!")
else:
    checkpoint_dir = matched_files[0].parent / "checkpoints"
    checkpoint_files = (
        list(checkpoint_dir.glob("*.ckpt")) if checkpoint_dir.exists() else []
    )

    if not checkpoint_files:
        print("No checkpoint files found in the checkpoint directory!")
    else:
        best_checkpoint = checkpoint_files[0] 
        best_rnn_model = RNNClassifier.load_from_checkpoint(
            best_checkpoint
        )
        print(best_rnn_model)

RNNClassifier(
  (model): RNN(
    (embedding): Embedding(16334, 300)
    (rnn): RNN(300, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=16, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=16, out_features=2, bias=True)
  )
  (metric): MulticlassAccuracy()
)


c:\Users\juinl\Documents\GitHub\sc4002-nlp-sentiment-classification\venv\lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.


In [20]:
import lightning as L
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, shuffle=True)

trainer = L.Trainer(accelerator="cpu")
trainer.test(best_rnn_model, test_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\juinl\Documents\GitHub\sc4002-nlp-sentiment-classification\venv\lib\site-packages\lightning\pytorch\trainer\setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
c:\Users\juinl\Documents\GitHub\sc4002-nlp-sentiment-classification\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\juinl\Documents\GitHub\sc4002-nlp-sentiment-classification\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7861163020133972
        test_loss           0.4762701690196991
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.4762701690196991, 'test_acc': 0.7861163020133972}]